## CNN

CNN has the same function as ordinary Neural network do. e.g.
- have same process of dot product
- have same differentiable score function
- same loss function(softmax, SVM)

**Q**: What changes and why CNN?

 **Reason**
   - **Reduce number of params**: In oridinary NN a single neuron of a layer is fully connected to all the neurons of the previous layers. In case of CIFAR-10, a single image has the shape[32,32,3] which means that a single neuron in the first layer needs 32*32*3=3072 params and a bias. since in nn the neurons don't share weights among them, the number of params for a single layer is then $s_j * 3073$, where $s_j$ means number of neurons. this semms a lot of params. moreover, We work with large imae than this. It's both wasteful and soon lead to overfitting.
   
  **Changes**: 
  - Since the input is image, they constrain the architecture based on their properties.
    - CNN neruons are 3 dimensional (witdth,height,depth). Images are input volume activations.
    - neurons are connected to only to a local region of the input volume(image)
    - final output layer (in case of cifar-10) is [1,1,10]. because the output volume of the last layer will be a vector of class scores. here, width=1,height=1 . so we will arrange the output vector along the depth dimension

     ![alt txt](images/cnn.PNG "cnn diagram")

## Layers of CNN

Three main types of layers.
- Convolutional Layer
- Pooling layer
- Fully-Connected Layer

Like ordinary NN we stack these layers to form a full ConvNet architecture. It's very important to understand the layers to work with CNN. Let's assume a CNN model for cifar-10 dataset.
- Input is a 3 dimensional image [32,32,3]
- CONV layer which takes the image as input: computes outputs of each neuron by dot product between the weights(size=keranlsize/filter size) and the local region on the image which is connected to filter. Final size [32,32,12] if we use 12 filters.
- ReLU applies elementwise activation function, such as max(0,x). **this leaves size of the volume unchanged**[32,32,12]
- POOL: this layer downsamples the volume spatially.
- FC: Although there can be more than one Fully-connected layer, the last one will compute the class scores. As the naem suggests the neurons of FC is connected to all the previous neurons. In case of cifar-10 image the result volume of the last FC layer will be 1X1X10. Here notice **class scores are along depth dimension**


Param(s) of layers:
- Convolutional and FC: have params { weights and biases}
- Pool and Relu: No params because they perform on fixed function

![alt txt](images/cnn1.PNG "cnn layers")
The activations of an example ConvNet architecture. The initial volume stores the raw image pixels (left) and the last volume stores the class scores (right). Each volume of activations along the processing path is shown as a column. Since it's difficult to visualize 3D volumes, we lay out each volume's slices in rows.The last layer volume holds the scores for each class, but here we only visualize the sorted top 5 scores.

## Convolutional Layer
Convolutional layers are the core building block of CNN. So it's important to understand the deep of it.
- **params**: Params of Conv layer is one or more 3 dimensional filters. e.g. a filter of size [5,5,3]. Look, the filters are small spatially but extends through the full depth of the image. During forward pass we convolve the filter spatially of the image but in full depth and compute **dot products between the entries of the filter and the input** at any position. We know dot product gives us a single scalar value. Thus after convolving throughtout the image we get a single scalar value for each convolution to a specific local region. Thus finally we end up with only a two dimensional feature map which contains the scalar values spatially. Now if we use more than one filter, we will get more activation map like this. This activations maps are kept along depth dimension. Thus the output of the convolution layer will be
[X,X,number_of_filters]. That means the depth is equal to the number of filters. I put X to the width and height position which will be discussed later. The early layers learns simple features and the later complex gradually.


**Local Connectivity**
- the filters are connected to only a local region of the input discussed earlier. This spatial extent of connectivity is called **receptive field**. But the extent of connectivity along the depth is full. In other words, connections are local in 2D and full along depth.

Example 1. For example, suppose that the input volume has size [32x32x3], (e.g. an RGB CIFAR-10 image). If the receptive field (or the filter size) is 5x5, then each neuron in the Conv Layer will have weights to a [5x5x3] region in the input volume, for a total of 5*5*3 = 75 weights (and +1 bias parameter). Notice that the extent of the connectivity along the depth axis must be 3, since this is the depth of the input volume.

Example 2. Suppose an input volume had size [16x16x20]. Then using an example receptive field size of 3x3, every neuron in the Conv Layer would now have a total of 3*3*20 = 180 connections to the input volume. Notice that, again, the connectivity is local in 2D space (e.g. 3x3), but full along the input depth (20).
